In [1]:
import os
from tqdm import tqdm
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from numpy import asarray

In [2]:
longitud, altura = 224, 224
modelo = './VGG16-Complicaciones/modelo_tf_vgg-exp2.h5'
pesos_modelo = './VGG16-Complicaciones/pesos_tf_vgg-exp2.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)

In [3]:
def Dataset_loader(DIR, RESIZE):
  IMG = []
  read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
  for IMAGE_NAME in tqdm(os.listdir(DIR)):
    PATH = os.path.join(DIR, IMAGE_NAME)
    _, ftype = os.path.splitext(PATH)
    if (ftype == ".jpg" or ftype=='.jpeg'):
      img = read(PATH)
      img = cv2.resize(img, (RESIZE, RESIZE))
      IMG.append(np.array(img) / 255.)
  return IMG

In [4]:
def Dataset_loader2(DIR, RESIZE):
  IMG = []
  for IMAGE_NAME in tqdm(os.listdir(DIR)):
    PATH = os.path.join(DIR, IMAGE_NAME)
    _, ftype = os.path.splitext(PATH)
    if (ftype == ".jpg" or ftype=='.jpeg'):
      image = Image.open(PATH).resize((RESIZE, RESIZE))
      data = asarray(image)
      IMG.append(np.array(data) / 255.)
  return IMG

In [53]:
train_benign = np.array(Dataset_loader('./DermMel/train_sep/NotMelanoma', altura))
train_malign = np.array(Dataset_loader('./DermMel/train_sep/Melanoma', altura))


benign_train_label = np.zeros(len(train_benign))
malign_train_label = np.ones(len(train_malign))

X_train = np.concatenate((train_benign, train_malign), axis = 0)
Y_train = np.concatenate((benign_train_label, malign_train_label), axis = 0)

100%|██████████| 5341/5341 [00:24<00:00, 217.74it/s]


In [55]:
y_pred = cnn.predict(X_train)
print("Done.")

Done.


In [87]:
#Segundo conjunto de imágenes
train_benign2 = np.array(Dataset_loader('./input/skin-cancer-malignant-vs-benign/train/benign', altura))
train_malign2 = np.array(Dataset_loader('./input/skin-cancer-malignant-vs-benign/train/malignant', altura))


benign_train_label2 = np.zeros(len(train_benign2))
malign_train_label2 = np.ones(len(train_malign2))

X_train2 = np.concatenate((train_benign2, train_malign2), axis = 0)
Y_train2 = np.concatenate((benign_train_label2, malign_train_label2), axis = 0)

100%|██████████| 1197/1197 [00:05<00:00, 229.76it/s]


In [88]:
y_pred4 = cnn.predict(X_train2)
print("Done.")

Done.


In [77]:
def ValidationTraining(y_pred,Y_test):
  print(len(y_pred))
  for j in range(50,0,-2):
      predictions = []
      truePositives = 0
      falsePositives = 0
      trueNegatives = 0
      falseNegatives = 0
      for i in range(len(y_pred)):
        if y_pred[i][0] > j/100:
          predictions.append(1)
          if Y_test[i]==1:
              truePositives+=1
          else: falsePositives+=1
        else:
          predictions.append(0)
          if Y_test[i]==0:
              trueNegatives+=1
          else: falseNegatives+=1

      sensitivity=truePositives/(truePositives+falseNegatives)
      especificity=trueNegatives/(trueNegatives+falsePositives)
      accuracy=accuracy_score(Y_test, predictions)
      print("ClassificationNumber:",j/100)
      print("Accuracy model: ",accuracy)
      print("Sensitivity: ",sensitivity)
      print("Especificity: ",especificity)
      print("TP",truePositives,"FN",falseNegatives,"TN",trueNegatives,"FP",falsePositives)
      print("-----------------------------------------------------------------")

In [78]:
ValidationTraining(y_pred,Y_train)

10682
ClassificationNumber: 0.5
Accuracy model:  0.8211009174311926
Sensitivity:  0.855457779442052
Especificity:  0.7867440554203333
TP 4569 FN 772 TN 4202 FP 1139
-----------------------------------------------------------------
ClassificationNumber: 0.48
Accuracy model:  0.8211945328590151
Sensitivity:  0.8633214753791425
Especificity:  0.7790675903388878
TP 4611 FN 730 TN 4161 FP 1180
-----------------------------------------------------------------
ClassificationNumber: 0.46
Accuracy model:  0.8222243025650627
Sensitivity:  0.8717468638831679
Especificity:  0.7727017412469575
TP 4656 FN 685 TN 4127 FP 1214
-----------------------------------------------------------------
ClassificationNumber: 0.44
Accuracy model:  0.8219434562815952
Sensitivity:  0.8775510204081632
Especificity:  0.7663358921550272
TP 4687 FN 654 TN 4093 FP 1248
-----------------------------------------------------------------
ClassificationNumber: 0.42
Accuracy model:  0.8219434562815952
Sensitivity:  0.884291331

In [89]:
ValidationTraining(y_pred4,Y_train2)

2637
ClassificationNumber: 0.5
Accuracy model:  0.7281001137656428
Sensitivity:  0.5739348370927319
Especificity:  0.85625
TP 687 FN 510 TN 1233 FP 207
-----------------------------------------------------------------
ClassificationNumber: 0.48
Accuracy model:  0.7341676147136899
Sensitivity:  0.5914786967418546
Especificity:  0.8527777777777777
TP 708 FN 489 TN 1228 FP 212
-----------------------------------------------------------------
ClassificationNumber: 0.46
Accuracy model:  0.7364429275692075
Sensitivity:  0.6015037593984962
Especificity:  0.8486111111111111
TP 720 FN 477 TN 1222 FP 218
-----------------------------------------------------------------
ClassificationNumber: 0.44
Accuracy model:  0.7402351156617368
Sensitivity:  0.6148705096073517
Especificity:  0.8444444444444444
TP 736 FN 461 TN 1216 FP 224
-----------------------------------------------------------------
ClassificationNumber: 0.42
Accuracy model:  0.7425104285172545
Sensitivity:  0.6274018379281537
Especificit

In [ ]:
#TESTING

In [6]:
#Para este modelo se ha encontrado que el mejor resultado de corte es 0.08
def ValidationTesting(y_pred,Y_test):
  classificationNumber=0.08
  predictions = []
  truePositives = 0
  falsePositives = 0
  trueNegatives = 0
  falseNegatives = 0
  for i in range(len(y_pred)):
    if y_pred[i][0] > classificationNumber:
      predictions.append(1)
      if Y_test[i]==1:
          truePositives+=1
      else: falsePositives+=1
    else:
      predictions.append(0)
      if Y_test[i]==0:
          trueNegatives+=1
      else: falseNegatives+=1

  sensitivity=truePositives/(truePositives+falseNegatives)
  especificity=trueNegatives/(trueNegatives+falsePositives)
  accuracy=accuracy_score(Y_test, predictions)
  print("ClassificationNumber:",classificationNumber)
  print("Accuracy model: ",accuracy)
  print("Sensitivity: ",sensitivity)
  print("Especificity: ",especificity)
  print("TP",truePositives,"FN",falseNegatives,"TN",trueNegatives,"FP",falsePositives)
  print("-----------------------------------------------------------------")

In [7]:
import random
random.seed(30)

test_benign = np.array(Dataset_loader('./DermMel/test/NotMelanoma', altura))
test_malign = np.array(Dataset_loader('./DermMel/test/Melanoma', altura))


benign_test_label = np.zeros(len(test_benign))
malign_test_label = np.ones(len(test_malign))

X_test = np.concatenate((test_benign, test_malign), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

random.shuffle(X_test)
random.shuffle(Y_test)

X_experimentsArray=np.array_split(X_test, 13)
Y_experimentsArray=np.array_split(Y_test, 13)

X_13E=[]
Y_13E=[]

print(len(test_benign),len(test_malign))
rangeNumber=0
for i in range(0,13):
    #100 imagenes benignas
    benign=(test_benign[rangeNumber:rangeNumber+100])
    zeros=(np.zeros(100))
    #100 imagenes malignas
    malign=(test_malign[rangeNumber:rangeNumber+100])
    ones=(np.ones(100))
    
    rangeNumber+=100
    print(rangeNumber)
    X_13E.append(np.concatenate((benign,malign),axis=0))
    Y_13E.append(np.concatenate((zeros,ones),axis=0))  


100%|██████████| 1781/1781 [00:06<00:00, 294.62it/s]


1780 1781
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [7]:
#Predicción (solo se ejecuta una vez)
y_pred2 = cnn.predict(X_test)
print("Done.")

Done.


In [8]:
print(len(X_test),len(Y_test),len(malign_test_label),len(benign_test_label),len(test_benign),len(test_malign))

3561 3561 1781 1780 1780 1781


In [9]:
ValidationTesting(y_pred2,Y_test)

ClassificationNumber: 0.08
Accuracy model:  0.7472620050547599
Sensitivity:  0.9679955081414935
Especificity:  0.5264044943820225
TP 1724 FN 57 TN 937 FP 843
-----------------------------------------------------------------


In [8]:
predictionsArray=[]
for experiment in X_13E:
   predictionsArray.append(cnn.predict(experiment)) 

In [9]:
print(len(X_13E),len(Y_13E),len(predictionsArray))
for i in range(len(predictionsArray)):
    print("Experimento ",i)
    ValidationTesting(predictionsArray[i],Y_13E[i])
    print("---------------------------------------------")

13 13 13
Experimento  0
ClassificationNumber: 0.08
Accuracy model:  0.81
Sensitivity:  0.98
Especificity:  0.64
TP 98 FN 2 TN 64 FP 36
-----------------------------------------------------------------
---------------------------------------------
Experimento  1
ClassificationNumber: 0.08
Accuracy model:  0.72
Sensitivity:  0.96
Especificity:  0.48
TP 96 FN 4 TN 48 FP 52
-----------------------------------------------------------------
---------------------------------------------
Experimento  2
ClassificationNumber: 0.08
Accuracy model:  0.75
Sensitivity:  0.97
Especificity:  0.53
TP 97 FN 3 TN 53 FP 47
-----------------------------------------------------------------
---------------------------------------------
Experimento  3
ClassificationNumber: 0.08
Accuracy model:  0.76
Sensitivity:  0.98
Especificity:  0.54
TP 98 FN 2 TN 54 FP 46
-----------------------------------------------------------------
---------------------------------------------
Experimento  4
ClassificationNumber: 0.

In [10]:
#Segundo dataset de testing
import random
random.seed(30)

test_benign2 = np.array(Dataset_loader2('./input/skin-cancer-malignant-vs-benign/test/benign', altura))
test_malign2 = np.array(Dataset_loader2('./input/skin-cancer-malignant-vs-benign/test/malignant', altura))

benign_test_label2 = np.zeros(len(test_benign2))
malign_test_label2 = np.ones(len(test_malign2))

X_test2 = np.concatenate((test_benign2, test_malign2), axis = 0)
Y_test2 = np.concatenate((benign_test_label2, malign_test_label2), axis = 0)

X_13E_2=[]
Y_13E_2=[]

print(len(test_malign2),len(test_benign2))
rangeNumber=0
for i in range(0,13):
    #20 imagenes benignas
    benign=(test_benign2[rangeNumber:rangeNumber+20])
    zeros=(np.zeros(20))
    #20 imagenes malignas
    malign=(test_malign2[rangeNumber:rangeNumber+20])
    ones=(np.ones(20))
    
    rangeNumber+=20
    print(rangeNumber)
    X_13E_2.append(np.concatenate((benign,malign),axis=0))
    Y_13E_2.append(np.concatenate((zeros,ones),axis=0))  

100%|██████████| 300/300 [00:00<00:00, 444.40it/s]


300 360
20
40
60
80
100
120
140
160
180
200
220
240
260


In [11]:
y_pred3 = cnn.predict(X_test2)
print("Done.")

Done.


In [12]:
ValidationTesting(y_pred3,Y_test2)

ClassificationNumber: 0.08
Accuracy model:  0.7772727272727272
Sensitivity:  0.9
Especificity:  0.675
TP 270 FN 30 TN 243 FP 117
-----------------------------------------------------------------


In [11]:
predictionsArray2=[]
for experiment in X_13E_2:
   predictionsArray2.append(cnn.predict(experiment)) 

In [12]:
print(len(X_13E_2[1]),len(Y_13E_2[1]),len(predictionsArray2[1]))
for i in range(len(predictionsArray2)):
    print("Experimento ",i)
    print(len(predictionsArray2[i]),len(Y_13E_2[i]))
    ValidationTesting(predictionsArray2[i],Y_13E_2[i])
    print("---------------------------------------------")

40 40 40
Experimento  0
40 40
ClassificationNumber: 0.08
Accuracy model:  0.725
Sensitivity:  0.95
Especificity:  0.5
TP 19 FN 1 TN 10 FP 10
-----------------------------------------------------------------
---------------------------------------------
Experimento  1
40 40
ClassificationNumber: 0.08
Accuracy model:  0.75
Sensitivity:  0.85
Especificity:  0.65
TP 17 FN 3 TN 13 FP 7
-----------------------------------------------------------------
---------------------------------------------
Experimento  2
40 40
ClassificationNumber: 0.08
Accuracy model:  0.8
Sensitivity:  1.0
Especificity:  0.6
TP 20 FN 0 TN 12 FP 8
-----------------------------------------------------------------
---------------------------------------------
Experimento  3
40 40
ClassificationNumber: 0.08
Accuracy model:  0.85
Sensitivity:  0.85
Especificity:  0.85
TP 17 FN 3 TN 17 FP 3
-----------------------------------------------------------------
---------------------------------------------
Experimento  4
40 40
